In [ ]:
import numpy as np
import pandas as pd
from glob import glob
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from skimage.graph import route_through_array

# Plots

In [ ]:
# make like other plots
from matplotlib.colors import LinearSegmentedColormap
colors = [(252/255,79/255,48/255), (229/255, 174/255,56/255), (0/255,143/255,213/255)]
cm = LinearSegmentedColormap.from_list('redblue', colors)

In [ ]:
# Load Sea/Land fixed data to create a mask
nc_f = 'data/sftlf_ARC-22_CCCma-CanESM2_rcp45_r1i1p1_CCCma-CanRCM4_r2_fx.nc'
nc = Dataset(nc_f, 'r')
#nc.variables.keys()
sftlf = nc['sftlf'][0, :, :]

In [ ]:
f = 'data/historic/day/sic_ARC-22_CCCma-CanESM2_historical_r1i1p1_CCCma-CanRCM4_r2_day_20010101-20051231.nc'

nc = Dataset(f, 'r')

weights = []
indices_len = []
times = []

times.extend(list(nc['time'][:]))

for i in range(nc['sic'].shape[0]):
    sic = nc['sic'][i, :, :]
    sic[np.where(sftlf!=0)] = -np.inf
    indices, weight = route_through_array(sic, [0,116], [265,231], geometric=True, fully_connected=True)
    weights.append(weight)
    indices_len.append(len(indices))

df = pd.DataFrame([weights, indices_len, times]).T
df.columns = ['weights', 'num_index', 'time']
df['date'] = pd.to_datetime(df.time, unit='D', origin='1949-12-01')
df['year'] = df['date'].dt.year
df['dayofyear'] = df.date.dt.dayofyear

In [ ]:
sic_max = nc['sic'][df.weights.idxmax(), :, :]
sic_max[np.where(sftlf!=0)] = -np.inf
indices, weight = route_through_array(sic_max, [0,116], [265,231], geometric=True, fully_connected=True)
indices = np.array(indices).T
path = np.zeros_like(sic)
path[indices[0], indices[1]] = 1

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
cb = ax.matshow(sic_max.T, origin='upper', cmap=cm, vmin=0, vmax=100)
i,j = np.where(path.T==1)
ax.scatter(j,i,s=10, c='black')
ax.axis('off')
plt.colorbar(cb, orientation='horizontal', fraction=0.046, pad=0.04)
plt.title('MCP weight = {}'.format(round(weight, 3)), loc='left', fontsize=20)
plt.suptitle('Sea Ice Concentration', ha='center', y=1.04, fontsize=22)
plt.tight_layout()
plt.savefig('plots/sic_ex1.png', dpi=72, pad_inches=0, bbox_inches="tight")
plt.show()

In [ ]:
sic_min = nc['sic'][df.weights.idxmin(), :, :]
sic_min[np.where(sftlf!=0)] = -np.inf
indices, weight = route_through_array(sic_min, [0,116], [265,231], geometric=True, fully_connected=True)
indices = np.array(indices).T
path = np.zeros_like(sic)
path[indices[0], indices[1]] = 1

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
cb = ax.matshow(sic_min.T, origin='upper', cmap=cm, vmin=0, vmax=100)
i,j = np.where(path.T==1)
ax.scatter(j,i,s=10, c='black')
ax.axis('off')
plt.colorbar(cb, orientation='horizontal', fraction=0.046, pad=0.04)
plt.title('MCP weight = {}'.format(round(weight, 3)), loc='left', fontsize=20)
plt.suptitle('Sea Ice Concentration', ha='center', y=1.04, fontsize=22)
plt.tight_layout()
plt.savefig('plots/sic_ex2.png', dpi=72, pad_inches=0, bbox_inches="tight")
plt.show()